**Reference Link:** [RAG Systems Essentials (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/rag-systems-essentials/lessons/60148017-hands-on-deep-dive-into-rag-evaluation-metrics-generator-metrics-i)

# Build a Simple RAG System

## Install OpenAI, and LangChain dependencies

In [0]:
!pip install langchain==0.3.10
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

## Install Chroma Vector DB and LangChain wrapper

In [0]:
!pip install langchain-chroma==0.1.4

## Install RAG Eval Libraries

In [0]:
!pip install ragas==0.2.8
!pip install deepeval==1.4.7

## Enter Open AI API Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [0]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Loading and Processing the Data

### Get the dataset

In [0]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1QkSY9W5RyaBnY8c5FLIsmpPVXoHTQ-fb/view?usp=sharing download it
# manually upload it on colab
!gdown 1QkSY9W5RyaBnY8c5FLIsmpPVXoHTQ-fb

### Load and Process JSON Documents

In [0]:
import pandas as pd

df = pd.read_csv('./rag_eval_docs.csv')
df

In [0]:
docs = df.to_dict(orient='records')
docs[:3]

In [0]:
from langchain.docstore.document import Document
processed_docs = []

for doc in docs:
    metadata = {
        "title": doc['title'],
        "id": doc['id'],
    }
    data = doc['context']
    processed_docs.append(Document(page_content=data, metadata=metadata))
processed_docs[:3]

## Index Document Chunks and Embeddings in Vector DB

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [0]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=processed_docs,
                                  collection_name='my_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [0]:
# load from disk
chroma_db = Chroma(persist_directory="./my_db",
                   collection_name='my_db',
                   embedding_function=openai_embed_model)

In [0]:
chroma_db

### Semantic Similarity based Retrieval

We use simple cosine similarity here and retrieve the top 3 similar documents based on the user input query

In [0]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                              search_kwargs={"k": 3, "score_threshold": 0.3})

In [0]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content))
        print()

In [0]:
query = "what is AI?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

In [0]:
query = "how do plants survive?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

## Build the RAG Pipeline

In [0]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer to the point based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [0]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

src_rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    chatgpt
        |
    StrOutputParser()
)

rag_chain_w_sources = (
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(response=src_rag_response_chain)
)

In [0]:
query = "What is AI?"
result = rag_chain_w_sources.invoke(query)
result

In [0]:
query = "How do plants survive?"
result = rag_chain_w_sources.invoke(query)
result

# Retriever Evaluation Metrics

![](https://i.imgur.com/5S4FhMB.png)

The retrieval process generally includes these steps:

- Convert the initial input query into an embedding using an embedding model of your choice (e.g., OpenAI's `text-embedding-3` model).
- Conduct a vector search with the embedded input on a vector database that holds your vectorized knowledge base, retrieving the top-K most "similar" document chunks.
- Optionally user a Reranker to rerank the retrieved results


Key Metrics to Evaluate here include:

- Contextual Precision
- Contextual Recall
- Contextual Relevancy

## Contextual Precision

The contextual precision metric measures your RAG pipeline's retriever by evaluating whether document chunks (nodes) in your `retrieval_context` that are relevant to the given `input` are ranked higher than irrelevant ones.

`deepeval`'s contextual precision metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a judge.

In `deepeval`, to use the ContextualPrecisionMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query
- `actual_output` : Actual LLM Response (not used in the computation)
- `expected_output` : Expected LLM Response (ground truth answer)
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/oVwrRAU.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                  like virtual assistants, robotics and autonomous vehicles."""

In [0]:
new_context = ['Machine Learning is the study of algorithms which learn with more data',
               'AI is known as Artificial Intelligence'] + retrieved_context
new_context

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualPrecisionMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metric = ContextualPrecisionMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

## Contextual Recall

The contextual recall metric measures the quality of your RAG pipeline's retriever by evaluating the extent of which the `retrieval_context` aligns with the `expected_output`.

`deepeval`'s contextual recall metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the ContextualRecallMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response (not used in the computation)
- `expected_output` : Expected LLM Response (ground truth answer)
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/PDbwuX5.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example 1:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                  like virtual assistants, robotics and autonomous vehicles."""

In [0]:
new_context = ['NVIDIA makes chips for AI', 'AI is an acronym for Artificial Intellence']
new_context

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRecallMetric
from deepeval import evaluate

test_case1 = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=retrieved_context
)

test_case2 = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metric = ContextualRecallMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case1, test_case2], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

In [0]:
print('Sucess:', result.test_results[1].metrics_data[0].success)
print('Score:', result.test_results[1].metrics_data[0].score)
print('Reason:', result.test_results[1].metrics_data[0].reason)

## Contextual Relevancy

The contextual relevancy metric measures the quality of your RAG pipeline's retriever by evaluating the overall relevance of the information presented in your `retrieval_context` for a given `input`.

`deepeval`'s contextual relevancy metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the ContextualRelevancyMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query
- `actual_output` : Actual LLM Response (not used in the computation)
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/VLKoEsI.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example 1:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
new_context = ['NVIDIA makes chips for AI', 'Google and Microsoft are battling out the market share for AI Chatbots'] + retrieved_context
new_context

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRelevancyMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metric = ContextualRelevancyMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

# Generator Evaluation Metrics

![](https://i.imgur.com/GaMHy7w.png)

The generation step, which comes after retrieval, generally includes:

- Building a prompt that combines the initial input with the context retrieved in the previous step.
- Feeding this prompt to your LLM, which produces the final generated response.


Key Metrics to Evaluate here include:

- Answer Relevancy
- Faithfulness
- Hallucination Check
- Custom LLM as a Judge (G-Eval)

## LLM-based Answer Relevancy - DeepEval

The answer relevancy metric measures the quality of your RAG pipeline's generator by evaluating how relevant the `actual_output` of your LLM application is compared to the provided `input`.

`deepeval`'s answer relevancy metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the AnswerRelevancyMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query
- `actual_output` : Actual LLM Response


![](https://i.imgur.com/GbNSCFC.png)



## Semantic Similarity based Answer Relevancy - RAGAS

DeepEval has bindings to Ragas which enables us to use the RAGASAnswerRelevancyMetric which focuses on assessing how pertinent the generated answer is to the given query using cosine similarity. A lower score is assigned to answers that are incomplete or contain redundant information and higher scores indicate better relevancy.

![](https://i.imgur.com/vq1ytZ3.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example - DeepEval:

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
)

metric = AnswerRelevancyMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

### Example - RAGAS:

In [0]:
from deepeval.metrics.ragas import RAGASAnswerRelevancyMetric

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
)

metric = RAGASAnswerRelevancyMetric(
    threshold=0.5,
    model="gpt-4o",
    embeddings=OpenAIEmbeddings()
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

## Faithfulness

The faithfulness metric measures the quality of your RAG pipeline's generator by evaluating whether the `actual_output` factually aligns with the contents of your `retrieval_context`.

`deepeval`'s faithfulness metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the FaithfulnessMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/OCSFPTb.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import FaithfulnessMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    retrieval_context=retrieved_context
)

metric = FaithfulnessMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

## Hallucination Check

The hallucination metric determines whether your LLM generates factually correct information by comparing the `actual_output` to the provided (human ground truth) `context`.

`deepeval`'s hallucination metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the HallucinationMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response
- `context` : Human Ground Truth Context Document Chunks (Nodes)


![](https://i.imgur.com/qyVBKU2.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
human_ground_truth_context = ["Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
                              "Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition."]
human_ground_truth_context

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import HallucinationMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    context=human_ground_truth_context
)

metric = HallucinationMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

In [0]:
ai_response = 'AI refers to machines mimicking human intelligence to produce cyborgs and electric sheep'

In [0]:
test_case = LLMTestCase(
    input=response['question'],
    actual_output=ai_response,
    context=human_ground_truth_context
)

metric = HallucinationMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

## Custom LLM as a Judge (G-Eval)

G-Eval is a framework that uses LLMs with chain-of-thoughts (CoT) to evaluate LLM outputs based on __ANY__ custom criteria.

The G-Eval metric is the most versatile type of metric `deepeval` has to offer, and is capable of evaluating almost any use case with good accuracy.

Here you are free to describe your custom evaluation process in detail using prompts in `evaluation_steps`.

In `deepeval`, to use the GEval, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response

You'll also need to supply any additional arguments such as `expected_output` and `context` if your evaluation criteria depends on these parameters.



![](https://i.imgur.com/IuODLKQ.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
ai_response = response['response']
ai_response

In [0]:
ai_response = 'AI refers to machines mimicking human intelligence, such as problem-solving and learning, and includes applications like electric sheep and cyborg kittens'
ai_response

In [0]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                  like virtual assistants, robotics and autonomous vehicles."""
human_answer

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import GEval
from deepeval import evaluate
from deepeval.test_case import LLMTestCaseParams

test_case = LLMTestCase(
    input=response['question'],
    actual_output=ai_response,
    expected_output=human_answer,
    retrieval_context=retrieved_context
)

metric = GEval(
    threshold=0.5,
    model="gpt-4o",
    name="RAG Fact Checker",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Create a list of statements from 'actual output'",
        "Validate if they are relevant and answers the given question in 'input', penalize if any statements are irrelevant",
        "Also Validate if they exist in 'expected output', penalize if any statements are missing or factually wrong",
        "Also validate if these statements are grounded in the 'retrieval context' and penalize if they are missing or factually wrong",
        "Finally also penalize if any statements seem to be invented or made up and do not make sense factually given the 'input' and 'retrieval context'"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT,
                       LLMTestCaseParams.ACTUAL_OUTPUT,
                       LLMTestCaseParams.EXPECTED_OUTPUT,
                       LLMTestCaseParams.RETRIEVAL_CONTEXT],
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)